In [24]:
#Essentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv 
import datetime as dt

#For scraping Reddit
import praw
from psaw import PushshiftAPI

#For EDA
import re
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.cluster import KMeans
import sklearn.cluster
import sklearn.decomposition
import matplotlib.colors
from textblob import TextBlob
from gensim.models import Word2Vec
import nltk
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import textstat
import gensim.corpora as corpora
from pprint import pprint
import gensim
from gensim.utils import simple_preprocess
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os

# Scraping data from reddit

In [2]:
# Before we can use praw, we have to create a Reddit instance do identify 
reddit = praw.Reddit(client_id='yyW8xfXOePfLhAE3sGhxXg', client_secret='ki5wj55ciBeoKhQghRtEJKS8QKJjPw', 
                     user_agent='Scraping')


# Mental Heatlh

In [135]:
#Thanks to dmarx for the clear code on pulling from PSAW! PSAW docs are surprisingly unclear.
#https://github.com/dmarx/psaw
api = PushshiftAPI()

#Start date we're beginning to pull posts from
start_epoch=int(dt.datetime(2015, 1, 1).timestamp())


#Pulling submissions from mentalhealth (both the title and body of a post)

gen_sub = list(api.search_submissions(after=start_epoch,
                            subreddit='mentalhealth',
                            filter=['title', 'id','score','link_flair_text','selftext'],
                            limit=100))


#Converting to dataframe
dfsub = pd.DataFrame([thing.d_ for thing in gen_sub])


C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [136]:
dfsub

,created_utc,id,score,selftext,title,created,link_flair_text
0,1663430008,xgq2e5,1,"When I remembered something embarrassing, disa...",Why do I hit myself when I’m angry,1.663444e+09,NaN
1,1663429739,xgpyf5,1,I’ve been mentally I’ll for so long now that j...,Does anyone know songs about not wanting to re...,1.663444e+09,NaN
2,1663429581,xgpw7p,1,[removed],Am I beyond repair?,1.663444e+09,Opinion / Thoughts
3,1663429164,xgpq15,1,"I am 18, and while I no longer fap to it like ...",Is there a problem with me liking furry/Pokemo...,1.663444e+09,Question
4,1663429149,xgppt8,1,"\nI thought we were getting along so well, cra...",Walking on a tightrope with my dad,1.663444e+09,Venting
...,...,...,...,...,...,...,...
95,1663401164,xggc25,1,My 4 yrs with company I was known to get it do...,I feel I am being messed with in a round about...,1.663416e+09,NaN
96,1663400621,xgg6m4,1,idc who and i dont need you to respond but the...,please someone talk to me,1.663415e+09,NaN
97,1663399985,xgg0ca,1,Maybe I’ll get into chess.,Hello,1.663414e+09,NaN
98,1663399711,xgfxji,1,I want to fucking break something because I fe...,I feel so lost,1.663414e+09,Venting


In [138]:
ids = list(dfsub.id)

In [139]:
len(ids)

100

In [140]:
new_ids = []
for i in ids:
    i = 't3_' + i
    new_ids.append(i)

In [141]:
#Pulling comments from relationship_advice

gen_com = api.search_comments(after=start_epoch, 
                            subreddit='mentalhealth',
                            filter=['body', 'link_id', 'id', 'score','submission'],
                            link_id = new_ids,
                            limit=350)

dfcom = pd.DataFrame([thing.d_ for thing in gen_com])

C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [146]:
dfcom

,body,created_utc,id,link_id,score,created
0,"A few things:\nMania or hypomania, required fo...",1663430571,iot6vvr,t3_xghgk6,1,1.663445e+09
1,"Between me and my best friend, who I hung out ...",1663430147,iot5uu7,t3_xgn4qt,1,1.663445e+09
2,I think it's important for people having a har...,1663429913,iot5alf,t3_xgieif,1,1.663444e+09
3,[removed],1663429801,iot50vq,t3_xgn4qt,1,1.663444e+09
4,Thank you for for sharing. A reminder: if you ...,1663429740,iot4vlg,t3_xgpyf5,1,1.663444e+09
...,...,...,...,...,...,...
274,please i need someone,1663400997,iorslov,t3_xgg6m4,1,1.663415e+09
275,Thank you for for sharing. A reminder: if you ...,1663400621,iors4p8,t3_xgg6m4,1,1.663415e+09
276,My advice to people suffering from depression ...,1663400229,iorrmsm,t3_xgfx8y,1,1.663415e+09
277,Thank you for for sharing. A reminder: if you ...,1663399711,iorqzy7,t3_xgfxji,1,1.663414e+09


In [147]:
len(dfcom.link_id.unique())

87

In [148]:
len(dfsub.id.unique())

100

In [150]:
dfcom.link_id.value_counts()

t3_xgkb2s    30
t3_xgg6m4    17
t3_xgjt3c    14
t3_xglxfh    13
t3_xghgk6    12
             ..
t3_xgmlls     1
t3_xglmoh     1
t3_xgl6nw     1
t3_xgl657     1
t3_xggc25     1
Name: link_id, Length: 87, dtype: int64

In [151]:
dfcom[dfcom['link_id'] == 't3_xgkb2s']

,body,created_utc,id,link_id,score,created
15,I'm not gonna say it's gonna be okay. I'm agai...,1663428881,iot2u0z,t3_xgkb2s,1,1.663443e+09
16,I've felt the same way but I promise you it ab...,1663428808,iot2nrp,t3_xgkb2s,1,1.663443e+09
24,I don’t feel anger. I feel resignation. I am i...,1663428254,iot1b68,t3_xgkb2s,1,1.663443e+09
26,I’m glad that’s your experience but that’s not...,1663428170,iot13zh,t3_xgkb2s,1,1.663443e+09
27,"Ah so it’s my fault actually, got it",1663428133,iot10rv,t3_xgkb2s,1,1.663443e+09
28,🎯🎯🎯🎯🎯,1663428112,iot0yy1,t3_xgkb2s,1,1.663443e+09
34,Fuck this. I hate answers like this because i...,1663427807,iot08r1,t3_xgkb2s,1,1.663442e+09
42,"Not everyone will have a stable, happy, or rel...",1663427626,iosztfw,t3_xgkb2s,1,1.663442e+09
56,It can get better tho. I am diagnosed with bip...,1663426324,ioswslw,t3_xgkb2s,1,1.663441e+09
59,"It gets better...for sure, it's not easy and i...",1663426023,iosw3b4,t3_xgkb2s,1,1.663440e+09


In [153]:
dfsub[dfsub['id'] == 'xgkb2s']

,created_utc,id,score,selftext,title,created,link_flair_text
52,1663415009,xgkb2s,1,that's all there is to it. i am farther behind...,it doesn't ever get better.,1.663429e+09,NaN


In [158]:
dfsub[dfsub['id'] == 'xgp6ow']

,created_utc,id,score,selftext,title,created,link_flair_text
10,1663427869,xgp6ow,1,I have a rant. I'm U.S. based ( just fyi) . Th...,Biggest problem was mental health help,1.663442e+09,NaN


In [154]:
dfcom[dfcom['link_id'] == 't3_xgp6ow']

,body,created_utc,id,link_id,score,created
32,Thank you for for sharing. A reminder: if you ...,1663427869,iot0e58,t3_xgp6ow,1,1.663442e+09
